In [ ]:
from my_trader import *
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib2
import time
import re
import getpass

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_driver.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_driver
#---------------------------
def render_text_contact(get_temp):
    flag = False
    trial =0
    result = pd.DataFrame()
    while not flag:
        try:
            get_temp = get_temp.text.split("\n")
            flag = True
        except Exception as e:
            print "Get text problem"
            if trial > 3:
                result = [np.NaN,np.NaN,np.NaN,np.NaN]
                return result
            driver.refresh()
            time.sleep(10+2*trial)
            trial +=1
            #get_temp = get_temp.get_attribute('innerHTML')
            #print ("!!!! verify: " + get_temp)
            #time.sleep(step_wait)
            #flag= True
            
    header=["name","position","location","industry"]
    count_header = 0
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        if ":" in get_temp[i]:
            #flag = True
            get = get_temp[i].split(":")
            result = result.append([get])

        else:
            result = result.append([[header[count_header],get_temp[i]]])
            count_header +=1

    result = result.reindex()    
    return result

def get_progress(name):
    mongodb = mongo("hoover")
    progress = pd.DataFrame(list(mongodb.db[name].find())).company_name
    progress = list(progress)
    return progress
#---------------------------




In [ ]:

#------PARAMETERS---------------

step_wait = 2
page_wait = 5
collection_name = "California Minority Food Growers"
list_row_number = 3  #input the list position of the list you want to crawl, starting 0

progress = get_progress(collection_name)


#flag initialize
from_beginning =  False

rebot_flag = True #default to be True to start with the program 

just_open_window = False  #default to be False to start with the program 
#-------------------------------

In [ ]:
username = raw_input()
password = getpass.getpass()

In [ ]:

if rebot_flag:  
    #reset flag
    rebot_flag = False
#===============OPEN BROWER=============================================
    driver=webdriver.Firefox()
    session_id =driver.session_id
    executor_url =driver.command_executor._url
    driver.implicitly_wait(5)

    driver.get('https://signon.onesource.com/Web/Logon/Signin.aspx')
    time.sleep(step_wait)
    driver.implicitly_wait(5)
    driver.find_element_by_id("username").clear()
    driver.find_element_by_id("username").send_keys(username)
    driver.find_element_by_id("password").clear()
    driver.find_element_by_id("password").send_keys(password)
    driver.find_element_by_id("btnSignIn").click()
    time.sleep(step_wait)
    driver.implicitly_wait(10)
    driver.find_element_by_link_text("Lists").click()
    time.sleep(step_wait)
    driver.implicitly_wait(10)

    # list selector

    list_ele = driver.find_elements_by_class_name("name-row")
    get = list_ele[list_row_number]
    get.find_element_by_tag_name("a").click()



    #page info

    current_page = driver.find_element_by_class_name("current-page")
    current_page=current_page.get_attribute('innerHTML')
    current_page = int(current_page)

    total_page = driver.find_element_by_class_name("total-page")
    total_page=total_page.get_attribute('innerHTML')
    total_page = int(total_page)
#================================================================================
if just_open_window:
    raise SystemExit(0)

driver.implicitly_wait(100)
current_page = driver.find_element_by_class_name("current-page")
current_page=current_page.get_attribute('innerHTML')
current_page = int(current_page)

# company selector
company_list = driver.find_elements_by_class_name("clickable")
name_list=[]
for i in range(len(company_list)):
    name_list.append(company_list[i].text)    

for i in name_list:
    #clear_output()

    #company_list = driver.find_elements_by_class_name("clickable")        
    #get_company = company_list[i]
    if i in progress:
        print (i+ " already in database, skipping")
        continue
    while True:
        try: 
            time.sleep(page_wait)
            driver.implicitly_wait(10)   
            get_company = driver.find_element_by_link_text(i)
            driver.execute_script("return arguments[0].scrollIntoView();", get_company)
            driver.execute_script("arguments[0].click();", get_company)
            #company_list[i].click()
            WebDriverWait(driver, 80).until(EC.presence_of_element_located((By.ID, "totalContacts")))
            time.sleep(step_wait)
            #get basic information
            info = {}
            address = driver.find_element_by_class_name("company-address").text
            company_name = driver.find_element_by_class_name("name").text
            try:
                website = driver.find_element_by_class_name("url").text
            except:
                print ("no website found")
            data = driver.find_elements_by_class_name("data-container")
            for i in range(len(data)):

                try:
                    label=data[i].find_element_by_class_name("data-label")
                except:
                    continue 
                    #label=data[i].find_element_by_class_name("data-label wide")
                value=data[i].find_elements_by_class_name("data-value")
                value = value[0].text
                label = label.text
                label = label.encode('ascii','ignore')
                info[label] = value

            info["address"] = address
            info["company_name"] = company_name
            info["website"] = website
            mongodb.db[collection_name].insert_one(info)
            print (company_name + ": basic information got")
            #get description about the company

            data_title = ["description","industry","highlights"]
            description=pd.DataFrame()
            for i in data_title:
                get_temp = driver.find_element_by_class_name("highlights")
                get_temp = get_temp.text.split("\n")
                #get_temp = get_temp.text.split(":")
            for i in range(len(get_temp)):
                get_temp[i] = get_temp[i].encode('ascii','ignore')
                get = get_temp[i].split(":")
                description = description.append([get])
            description = description.set_index(0)
            description=json.loads(description.to_json()).values()[0]
            mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":description},upsert=True)
            print (company_name + ": description information got")

        except:
            print ("---------------")
            print e
            print ("---------------")
            if trial > 3:
                break
            driver.back()
            driver.forward()
            trial +=1            

    #Get contacts
    contacts_flag = False
    contact_trial = 0 
    while not contacts_flag:
        try:
            contacts = driver.find_element_by_id("totalContacts").text
            contacts = int(contacts)
            contacts_flag = True
        except:
            #print ("contact number didn't come out yet")
            continue
    while True:
        try:
            if contacts != 0:
                try:
                    driver.find_element_by_id("company_contacts").click()
                except:
                    driver.execute_script("arguments[0].click();", driver.find_element_by_id("company_contacts"))
                time.sleep(step_wait)
                WebDriverWait(driver, step_wait).until(EC.presence_of_element_located((By.CLASS_NAME, "detail-container")))                                   
                contacts = driver.find_elements_by_class_name("detail-container")

                contacts = driver.find_elements_by_class_name("search-result-container")
                result = []
                for i in range(len(contacts)):

                    temp = render_text_contact(contacts[i])
                    temp = temp.set_index(temp[0])
                    temp = temp.drop(0,axis=1)
                    result.append(json.loads(temp.to_json()).values())
                mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":{"contact":result}},upsert=True)
                print (company_name + ": contacts information got")
                driver.back()
            else:
                print ("no contact available")

            # page wait time   
            progress.append(company_name)
            driver.back()
        except:
            print ("---------------")
            print e
            print ("---------------")
            if trial > 3:
                break
            driver.back()
            driver.forward()
            trial +=1

# next page
if from_beginning:
    if total_page > current_page:
        next_page = driver.find_element_by_class_name("next-page")
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(page_wait)
        driver.implicitly_wait(10) 
    else:
        break
else:
    if 1 < current_page:
        last_page = driver.find_element_by_class_name("last-page")
        driver.execute_script("arguments[0].click();", last_page)
        time.sleep(page_wait)
        driver.implicitly_wait(10) 
    else:
        break

#================================================    
#Getting hoover data
mongodb = mongo("hoover")

hoover_data = pd.DataFrame(list(mongodb.db[collection_name].find())).drop("",axis = 1)
hoover_data=hoover_data.set_index("company_name")
hoover_data=hoover_data.loc[:,"contact"].dropna()
#Getting hoover contact data
hoover_contacts = pd.DataFrame()
for i in range(len(hoover_data)):
    for j in hoover_data.iloc[i]:
        temp = pd.DataFrame(j,index=[hoover_data.index[i]])
        hoover_contacts = hoover_contacts.append(temp)

hoover_contacts.index.name = "company_name"
hoover_data.to_csv(collection_name + ".csv")
hoover_contacts.to_csv(collection_name+"_contacts.csv")